# Product Mart Queries

In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

In [ ]:
PROJ_ROOT = Path().resolve().parents[3]
env_file_dir = PROJ_ROOT / '.env'
_ = load_dotenv(env_file_dir, verbose=True)

## About

Develop **product mart** queries using DBT's `intermediate` models.

All query results are per product.

### Notes

1. This notebook supports <kbd>Run</kbd> > <kbd>Run All Cells</kbd>.

## User Inputs

In [ ]:
#

In [ ]:
engine = create_engine(
    URL(
        drivername="driver",
        account=os.getenv("UPLIMIT_SNOWFLAKE_ACCOUNT"),
        user=os.getenv("UPLIMIT_SNOWFLAKE_USER"),
        password=os.getenv("UPLIMIT_SNOWFLAKE_PASS"),
        warehouse=os.getenv("UPLIMIT_SNOWFLAKE_WAREHOUSE"),
        role=os.getenv("UPLIMIT_SNOWFLAKE_ROLE"),
        database=os.getenv("UPLIMIT_SNOWFLAKE_DB_NAME"),
        schema=os.getenv("UPLIMIT_SNOWFLAKE_SCHEMA"),
    )
)

## Connect

Load Jupyter SQL extension

In [ ]:
%load_ext sql

Set the maximum number of rows to be displayed to `None` (show all rows)

In [ ]:
%config SqlMagic.displaylimit = None

Connect to DuckDB database

In [ ]:
%sql engine --alias connection

## Queries

### Daily Page Views and Orders per Product

### `marts/products/daily/fct_products_daily`

In [ ]:
%%sql
/* get daily product page views and orders */
WITH daily_product_summary_sorted AS (
    SELECT pn.name AS product_name,
           dpv.created_at_date,
           dpv.num_page_views,
           dpv.num_orders
    FROM int_products_daily_totals dpv
    -- join to get product name instead of product ID
    INNER JOIN stg_postgres_products pn USING (product_id)
    -- sort to show top-performing prducts first
    ORDER BY created_at_date ASC, num_orders DESC
)
SELECT *
FROM daily_product_summary_sorted

### Total Traffic, Conversions (Purchases), Cart Abandonment Rate & Average Time on Page, Per Product

### `marts/products/fct_products`

In [ ]:
%%sql
/* get average time spent viewing product page */
WITH product_page_timings AS (
    SELECT *
    FROM int_products_page_viewing_time_averaged
),
/* get number of times product was added to cart */
product_carts AS (
    SELECT product_id,
           num_carts
    FROM int_products_purchase_abandoned_cart_sessions_summed
),
/* get total product page views (traffic) and purchases */
product_traffic_purchases AS (
    SELECT product_id,
           num_purchases,
           num_page_views,
           -- calculate total number of sessions (with or without a purchase)
           -- in which a product page was viewed
           (
               num_non_purchase_page_view_sessions
               +num_purchase_page_view_sessions
           ) AS num_page_view_sessions
    FROM int_events_sessions_aggregated_to_product
),
/* combine product metrics and add ranks by page views and purchases */
product_summary AS (
    SELECT pn.name AS product_name,
           pt.num_page_view_sessions,
           pt.num_page_views,
           pt.num_purchases,
           -- rank products by traffic (page views)
           RANK() OVER(ORDER BY num_page_views DESC) AS rank_traffic,
           -- rank products by purchases
           RANK() OVER(ORDER BY num_purchases DESC) AS rank_purchases,
           -- count total number of products with metrics
           COUNT(*) OVER() AS num_products,
           -- calculate cart abandonment rate
           -- car.cart_abandonment_rate,
           1-(pt.num_purchases/pc.num_carts) AS cart_abandonment_rate,
           -- get average time on product page
           pv.avg_time_on_page_seconds
    FROM product_carts pc
    INNER JOIN product_page_timings pv USING (product_id)
    -- INNER JOIN product_cart_abandonment_rates car USING (product_id)
    INNER JOIN product_traffic_purchases pt USING (product_id)
    -- join to get product name instead of product ID
    INNER JOIN stg_postgres_products pn USING (product_id)
),
/* add product indicators for high traffic (page views) and low conversions
(purchases) */
product_performance_indicators AS (
    SELECT * EXCLUDE(num_products),
           -- add high-traffic indicator per product
           (
               CASE WHEN rank_traffic <= 10 THEN True ELSE False END
           ) AS is_high_traffic,
           -- add low-conversion indicator per product
           (
               CASE
                   WHEN rank_purchases >= num_products-10
                   THEN True
                   ELSE False
               END
           ) AS is_low_conversions
    FROM product_summary
)
SELECT *
FROM product_performance_indicators

## Disconnect

Close connection

In [ ]:
%sql --close connection